In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os
from tqdm import tqdm
from os.path import join as pjoin
import random
import sys # For error exit

# --- Configuration ---
POSE_FEATURES_DIM = 263  # Original dimension
LATENT_DIM = 64          # Target latent dimension (Hyperparameter to tune!)
LEARNING_RATE = 1e-4
WEIGHT_DECAY = 1e-5
BATCH_SIZE = 128         # Adjust based on GPU memory
EPOCHS = 100             # Adjust as needed
DATA_DIR = "..\\HumanML3D" # Directory containing new_joint_vecs and train/val lists
SAVE_DIR = "model_saves/autoencoder" # Directory to save checkpoints
MODEL_NAME = "pose_ae" # Base name for saved files
NORMALIZATION_STATS_PATH = "pose_stats.npz" # Path to your mean/std file
VAL_SETTING = "val"      # Assumes you have 'val.txt' for validation
TRAIN_SETTING = "train"  # Assumes you have 'train.txt'
USE_PERCENTAGE = 1.0     # Use 100% of train/val data listed

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Ensure save directory exists
os.makedirs(SAVE_DIR, exist_ok=True)
BEST_MODEL_SAVE_PATH = pjoin(SAVE_DIR, f"{MODEL_NAME}_best.pth")
FINAL_MODEL_SAVE_PATH = pjoin(SAVE_DIR, f"{MODEL_NAME}_final.pth")


Using device: cuda


In [5]:

# --- Autoencoder Model ---
class Encoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, latent_dim)
        )
    def forward(self, x):
        return self.net(x)

class Decoder(nn.Module):
    def __init__(self, latent_dim, output_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, output_dim)
        )
    def forward(self, x):
        return self.net(x)

class Autoencoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super().__init__()
        self.encoder = Encoder(input_dim, latent_dim)
        self.decoder = Decoder(latent_dim, input_dim)

    def forward(self, x):
        latent = self.encoder(x)
        reconstruction = self.decoder(latent)
        return reconstruction

# --- Dataset ---
class PoseDatasetAE(Dataset):
    def __init__(self, data_dir, setting, norm_stats_path, use_percentage=1.0):
        self.data_dir = data_dir
        pose_vec_dir = pjoin(data_dir, "new_joint_vecs")
        self.pose_files = []
        self.mean = None
        self.std = None

        list_file_path = pjoin(data_dir, f"{setting}.txt")
        if not os.path.exists(list_file_path):
             print(f"Error: List file not found at {list_file_path}")
             # Depending on required functionality, either raise error or try fallback
             raise FileNotFoundError(f"Required list file missing: {list_file_path}")
             # Or try fallback:
             # print(f"Warning: {list_file_path} not found. Trying train.txt")
             # list_file_path = pjoin(data_dir, "train.txt")

        with open(list_file_path, "r") as f:
            file_list = [line.strip() for line in f.readlines()]

        print(f"Found {len(file_list)} entries in {setting}.txt")

        valid_files_count = 0
        for file_id in file_list:
             pose_path = pjoin(pose_vec_dir, f"{file_id}.npy")
             if os.path.exists(pose_path): # Check if file actually exists
                 self.pose_files.append(pose_path)
                 valid_files_count += 1
             else:
                 print(f"Warning: Pose file not found for ID {file_id} at {pose_path}")

        print(f"Found {valid_files_count} existing pose files for {setting}.")
        if valid_files_count == 0:
            raise ValueError(f"No valid pose files found for setting '{setting}'. Check paths.")


        if 0 < use_percentage < 1.0:
            random.shuffle(self.pose_files)
            num_samples = int(len(self.pose_files) * use_percentage)
            self.pose_files = self.pose_files[:num_samples]
            print(f"Using {len(self.pose_files)} files ({use_percentage*100:.1f}%) for {setting}.")


        # Load normalization stats
        try:
            stats = np.load(norm_stats_path)
            self.mean = torch.tensor(stats['mean'], dtype=torch.float32).unsqueeze(0) # (1, D)
            self.std = torch.tensor(stats['std'], dtype=torch.float32).unsqueeze(0)   # (1, D)
            # Add epsilon to std to prevent division by zero or large values
            self.std = torch.where(self.std < 1e-6, torch.tensor(1e-6, dtype=torch.float32), self.std)
            print(f"Normalization stats loaded from {norm_stats_path}")
        except FileNotFoundError:
            print(f"Error: Normalization file not found at {norm_stats_path}")
            print("Cannot proceed without normalization stats.")
            sys.exit(1) # Exit if stats are missing

    def __len__(self):
        return len(self.pose_files)

    def __getitem__(self, index):
        pose_path = self.pose_files[index]
        try:
            # Ensure file exists right before loading (paranoia check)
            if not os.path.exists(pose_path):
                 print(f"Error: File disappeared before loading? {pose_path}")
                 return None
            pose_data = np.load(pose_path) # Shape: (frames, features)
            # Basic check for valid data (e.g., not empty)
            if pose_data.shape[0] == 0 or pose_data.shape[1] != POSE_FEATURES_DIM:
                 print(f"Warning: Invalid shape {pose_data.shape} in {pose_path}. Skipping.")
                 return None
        except Exception as e:
            print(f"Error loading or validating {pose_path}: {e}")
            return None # Handle potential loading errors in collate_fn

        pose_tensor = torch.tensor(pose_data, dtype=torch.float32)

        # Normalize
        pose_normalized = (pose_tensor - self.mean) / self.std

        return pose_normalized # Return the whole sequence

# --- Custom Collate Function ---
def collate_ae(batch):
    # Filter out None items resulting from loading errors or invalid shapes
    batch = [item for item in batch if item is not None and item.ndim == 2 and item.shape[0] > 0]
    if not batch:
        return None
    # Concatenate all frames from the batch sequences
    try:
        all_frames = torch.cat(batch, dim=0) # Shape: (total_frames_in_batch, features)
    except Exception as e:
        print(f"Error during concatenation: {e}")
        # Optionally print shapes of items in batch for debugging
        # for i, item in enumerate(batch): print(f"Item {i} shape: {item.shape}")
        return None
    return all_frames

In [ ]:

# --- Training Setup ---
model = Autoencoder(POSE_FEATURES_DIM, LATENT_DIM).to(device)
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
criterion = nn.MSELoss() # Reconstruction loss

# Create Datasets and DataLoaders
try:
    train_dataset = PoseDatasetAE(DATA_DIR, TRAIN_SETTING, NORMALIZATION_STATS_PATH, use_percentage=USE_PERCENTAGE)
    val_dataset = PoseDatasetAE(DATA_DIR, VAL_SETTING, NORMALIZATION_STATS_PATH, use_percentage=USE_PERCENTAGE) # Use same percentage for val? Or 1.0?
except (FileNotFoundError, ValueError) as e:
    print(f"Error creating datasets: {e}")
    sys.exit(1)

# Use num_workers > 0 for faster data loading if not on Windows/debugging
# Set persistent_workers=True if using num_workers > 0 for efficiency between epochs
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_ae, num_workers=0, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_ae, num_workers=0, pin_memory=True)


# --- Training Loop ---
print("Starting Autoencoder Training...")
best_val_loss = float('inf')

for epoch in range(EPOCHS):
    # --- Training Phase ---
    model.train()
    total_train_loss = 0.0
    num_train_batches = 0
    pbar_train = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [Train]", leave=False)

    for batch_frames in pbar_train:
        if batch_frames is None: continue # Skip empty batches

        batch_frames = batch_frames.to(device, non_blocking=True) # Use non_blocking with pin_memory
        num_train_batches += 1

        reconstructions = model(batch_frames)
        loss = criterion(reconstructions, batch_frames)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()
        pbar_train.set_postfix({"Loss": f"{loss.item():.6f}"})

    avg_train_loss = total_train_loss / num_train_batches if num_train_batches > 0 else 0

    # --- Validation Phase ---
    model.eval()
    total_val_loss = 0.0
    num_val_batches = 0
    pbar_val = tqdm(val_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [Val]", leave=False)

    with torch.no_grad():
        for batch_frames in pbar_val:
            if batch_frames is None: continue # Skip empty batches

            batch_frames = batch_frames.to(device, non_blocking=True)
            num_val_batches += 1

            reconstructions = model(batch_frames)
            loss = criterion(reconstructions, batch_frames) # Use same MSE loss

            total_val_loss += loss.item()
            pbar_val.set_postfix({"Loss": f"{loss.item():.6f}"})

    avg_val_loss = total_val_loss / num_val_batches if num_val_batches > 0 else float('inf')

    print(f"Epoch {epoch+1}/{EPOCHS} -> Train Loss: {avg_train_loss:.6f}, Val Loss: {avg_val_loss:.6f}")

    # --- Save Model Checkpoints ---
    # Save based on best validation loss
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': best_val_loss,
            'input_dim': POSE_FEATURES_DIM,
            'latent_dim': LATENT_DIM,
        }, BEST_MODEL_SAVE_PATH)
        print(f"    -> New best validation loss ({best_val_loss:.6f}). Saved best model checkpoint.")

    # Optional: Save periodic checkpoints regardless of validation loss
    # if (epoch + 1) % 10 == 0:
    #      periodic_path = f"{PERIODIC_SAVE_PREFIX}{epoch+1}.pth"
    #      torch.save({
    #          'epoch': epoch + 1,
    #          'model_state_dict': model.state_dict(),
    #          'input_dim': POSE_FEATURES_DIM,
    #          'latent_dim': LATENT_DIM,
    #      }, periodic_path)
    #      print(f"    -> Saved periodic checkpoint to {periodic_path}")


# --- Save Final Model ---
torch.save({
    'model_state_dict': model.state_dict(),
    'input_dim': POSE_FEATURES_DIM,
    'latent_dim': LATENT_DIM,
 }, FINAL_MODEL_SAVE_PATH)
print(f"Training finished. Final model saved to {FINAL_MODEL_SAVE_PATH}")
print(f"Best validation loss achieved: {best_val_loss:.6f} (Model saved to {BEST_MODEL_SAVE_PATH})")

Found 23384 entries in train.txt
Found 23384 existing pose files for train.
Normalization stats loaded from pose_stats.npz
Found 1460 entries in val.txt
Found 1460 existing pose files for val.
Normalization stats loaded from pose_stats.npz
Starting Autoencoder Training...


Epoch 1/100 -> Train Loss: 0.802107, Val Loss: 0.540647
    -> New best validation loss (0.540647). Saved best model checkpoint.


Epoch 2/100 -> Train Loss: 0.451959, Val Loss: 0.397321
    -> New best validation loss (0.397321). Saved best model checkpoint.


Epoch 3/100 -> Train Loss: 0.363768, Val Loss: 0.335546
    -> New best validation loss (0.335546). Saved best model checkpoint.


Epoch 4/100 -> Train Loss: 0.313310, Val Loss: 0.296996
    -> New best validation loss (0.296996). Saved best model checkpoint.


...

Epoch 97/100 -> Train Loss: 0.069961, Val Loss: 0.070480


Epoch 98/100 -> Train Loss: 0.069587, Val Loss: 0.069839
    -> New best validation loss (0.069839). Saved best model checkpoint.


Epoch 99/100 -> Train Loss: 0.069368, Val Loss: 0.069542
    -> New best validation loss (0.069542). Saved best model checkpoint.


Epoch 100/100 -> Train Loss: 0.069058, Val Loss: 0.069141
    -> New best validation loss (0.069141). Saved best model checkpoint.
Training finished. Final model saved to model_saves/autoencoder\pose_ae_final.pth
Best validation loss achieved: 0.069141 (Model saved to model_saves/autoencoder\pose_ae_best.pth)


: 